In [10]:
import pandas as pd
import os

# 主表A的路径
main_table_path = 'combine_news.xlsx'  # 替换为主表A的实际路径

# 文件夹路径
folder_path = "D:/mycodelife/workshop/fake_finance/course_task"  # 替换为文件夹的实际路径

# 输出总表路径
output_path = 'combine_news.xlsx'  # 替换为输出文件的实际路径

# 读取主表A
df_main = pd.read_excel(main_table_path,sheet_name= 0 ,dtype={'股票代码':str})

# 确保主码列名正确
main_keys = ['股票代码', '新闻发布时间']

# 检查主表是否包含主码列
if not set(main_keys).issubset(df_main.columns):
    raise ValueError(f"主表 A 缺少主码列：{set(main_keys) - set(df_main.columns)}")

# 初始化结果表
result = df_main.copy()

# 遍历文件夹中的所有 Excel 文件
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        
        # 读取当前文件
        df = pd.read_excel(file_path,dtype={'股票代码':str})

        # 检查当前表是否包含主码列
        if not set(main_keys).issubset(df.columns):
            print(f"文件 {file_name} 缺少主码列，已跳过。")
            continue

        # 按主码合并
        result = pd.merge(result, df, on=main_keys, how='left', suffixes=('', f'_{file_name}'))

# 将结果输出到总表
result.to_excel(output_path, index=False)

print(f"匹配完成，结果已输出到 {output_path}")


匹配完成，结果已输出到 combine_news.xlsx


In [ ]:
import numpy as np
import pandas as pd

data  = pd.read_excel('combine_news.xlsx',dtype={'股票代码':str})
# Define the columns of interest for titles and comments
title_cols = ["标题中性分数 (t+0)", "标题消极分数 (t+0)", "标题积极分数 (t+0)"]
comment_cols = ["评论中性分数 (t+0)", "评论消极分数 (t+0)", "评论积极分数 (t+0)"]

# Extract the columns of interest along with neighboring ones for filling
related_cols = [
    "标题中性分数 (t+0)", "标题消极分数 (t+0)", "标题积极分数 (t+0)",
    "标题中性分数 (t+1)", "标题消极分数 (t+1)", "标题积极分数 (t+1)",
    "标题中性分数 (t-1)", "标题消极分数 (t-1)", "标题积极分数 (t-1)",
    "评论中性分数 (t+0)", "评论消极分数 (t+0)", "评论积极分数 (t+0)",
    "评论中性分数 (t+1)", "评论消极分数 (t+1)", "评论积极分数 (t+1)",
    "评论中性分数 (t-1)", "评论消极分数 (t-1)", "评论积极分数 (t-1)",
]

# Subset the related columns
data_related = data[related_cols]

# Fill missing values for each column of interest
def fill_with_nearest(row, main_col, neighbors):
    if not np.isnan(row[main_col]):
        return row[main_col]
    for neighbor in neighbors:
        if not np.isnan(row[neighbor]):
            return row[neighbor]
    return np.nan

# Columns for which we need to find neighbors
title_neighbors = {
    "标题中性分数 (t+0)": ["标题中性分数 (t+1)", "标题中性分数 (t-1)"],
    "标题消极分数 (t+0)": ["标题消极分数 (t+1)", "标题消极分数 (t-1)"],
    "标题积极分数 (t+0)": ["标题积极分数 (t+1)", "标题积极分数 (t-1)"],
}
comment_neighbors = {
    "评论中性分数 (t+0)": ["评论中性分数 (t+1)", "评论中性分数 (t-1)"],
    "评论消极分数 (t+0)": ["评论消极分数 (t+1)", "评论消极分数 (t-1)"],
    "评论积极分数 (t+0)": ["评论积极分数 (t+1)", "评论积极分数 (t-1)"],
}

# Apply the filling logic
for col, neighbors in {**title_neighbors, **comment_neighbors}.items():
    data_related[col] = data_related.apply(lambda row: fill_with_nearest(row, col, neighbors), axis=1)

# Combine the filled columns with the original data
result_data = pd.concat([data.drop(columns=related_cols), data_related[title_cols + comment_cols]], axis=1)

# Show the first few rows of the processed data
result_data.head()

result_data.to_excel("ml_without_label.xlsx")

C:\Users\86198\AppData\Local\Temp\ipykernel_18540\3893153755.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_related[col] = data_related.apply(lambda row: fill_with_nearest(row, col, neighbors), axis=1)


In [4]:
import pandas as pd
import numpy as np

# Step 1: 读取Excel文件
file_path = "combine_news.xlsx"  # 替换为您的文件路径
data = pd.read_excel(file_path)

# Step 2: 填充逻辑
# 定义需要处理的列及邻近列范围
new_title_neighbors = {
    "标题中性分数 (t+0)": [f"标题中性分数 (t{n:+d})" for n in range(-4, 4) if n != 0],
    "标题消极分数 (t+0)": [f"标题消极分数 (t{n:+d})" for n in range(-4, 4) if n != 0],
    "标题积极分数 (t+0)": [f"标题积极分数 (t{n:+d})" for n in range(-4, 4) if n != 0],
}
new_comment_neighbors = {
    "评论中性分数 (t+0)": [f"评论中性分数 (t{n:+d})" for n in range(-4, 4) if n != 0],
    "评论消极分数 (t+0)": [f"评论消极分数 (t{n:+d})" for n in range(-4, 4) if n != 0],
    "评论积极分数 (t+0)": [f"评论积极分数 (t{n:+d})" for n in range(-4, 4) if n != 0],
}

# 定义填充函数：如果目标列为空，依次从邻近列寻找非空值填充
def fill_with_nearest(row, main_col, neighbors):
    if not np.isnan(row[main_col]):
        return row[main_col]
    for neighbor in neighbors:
        if neighbor in row and not np.isnan(row[neighbor]):
            return row[neighbor]
    return np.nan

# 对每个t+0列进行填充处理
for col, neighbors in {**new_title_neighbors, **new_comment_neighbors}.items():
    data[col] = data.apply(lambda row: fill_with_nearest(row, col, neighbors), axis=1)

# Step 3: 增加标签
# 计算每个(t+0)列的均值（忽略NaN），生成标签列
for col in [
    "标题中性分数 (t+0)", "标题消极分数 (t+0)", "标题积极分数 (t+0)",
    "评论中性分数 (t+0)", "评论消极分数 (t+0)", "评论积极分数 (t+0)"
]:
    mean_value = data[col].mean(skipna=True)
    data[f"{col} 标签"] = data[col].apply(lambda x: 1 if x > mean_value else (0 if not np.isnan(x) else np.nan))

# Step 4: 输出最终数据
# 保留所有非 (t+n) 相关列，以及处理后的 t+0 列和其标签
t0_columns = [
    "标题中性分数 (t+0)", "标题消极分数 (t+0)", "标题积极分数 (t+0)",
    "评论中性分数 (t+0)", "评论消极分数 (t+0)", "评论积极分数 (t+0)"
]
label_columns = [f"{col} 标签" for col in t0_columns]
non_tn_columns = [col for col in data.columns if not any(suffix in col for suffix in ["(t+", "(t-", "(t0)"])]

final_columns = non_tn_columns + t0_columns + label_columns
final_data = data[final_columns]

# Step 5: 保存到Excel文件
output_file_path = "processed_with_labels.xlsx"  # 替换为您想保存的路径
final_data.to_excel(output_file_path, index=False)


如果评论中性分数为空，则用新闻真实性填充

In [1]:
import pandas as pd

# Step 1: 读取处理好的数据
file_path = "processed_with_labels.xlsx"  # 替换为您的文件路径
data = pd.read_excel(file_path)

# Step 2: 填充缺失值
# 如果“评论中性分数 (t+0) 标签”列中有缺失值，用“真实性（虚假1真实0）”的值填充
data["评论中性分数 (t+0) 标签"] = data["评论中性分数 (t+0) 标签"].fillna(data["真实性（虚假1真实0）"])

# Step 3: 保存处理后的数据
output_file_path = "filled_labels.xlsx"  # 替换为您想保存的路径
data.to_excel(output_file_path, index=False)


删除列“新闻文本情绪”之类都是空的

In [2]:
import pandas as pd

# Step 1: 读取文件
file_path = "filled_labels.xlsx"  # 替换为您的文件路径
data = pd.read_excel(file_path)

# Step 2: 删除特定列全为空的行
# 指定需要检查的列
columns_to_check = ["新闻内容中性分数", "新闻内容积极分数", "新闻内容消极分数"]

# 删除所有指定列都为空的行
data = data.dropna(subset=columns_to_check, how='all')

# Step 3: 保存处理后的数据
output_file_path = "filtered_data.xlsx"  # 替换为您想保存的路径
data.to_excel(output_file_path, index=False)


将日期转化为数值，将来源分类

In [3]:
import pandas as pd

# Step 1: 读取文件
file_path = "filtered_data.xlsx"  # 替换为您的文件路径
data = pd.read_excel(file_path)

# Step 2: 处理“新闻实际来源”
# 统计“新闻实际来源”的唯一值
unique_sources = data["新闻实际来源"].dropna().unique()

# 创建来源类别映射表
source_mapping = {source: idx for idx, source in enumerate(unique_sources)}

# 新增“来源类别”列
data["来源类别"] = data["新闻实际来源"].map(source_mapping)

# Step 3: 处理“新闻发布时间”
# 转换为日期格式
data["新闻发布时间"] = pd.to_datetime(data["新闻发布时间"])

# 找到最早的日期并设为基准
earliest_date = data["新闻发布时间"].min()

# 新增“发布时间”列，计算日期偏移（以天为单位）
data["发布时间"] = (data["新闻发布时间"] - earliest_date).dt.days

# Step 4: 保存处理后的数据
output_file_path = "processed_data.xlsx"  # 替换为您想保存的路径
data.to_excel(output_file_path, index=False)


填充基本面数据

In [4]:
import pandas as pd

# Step 1: 读取文件
file_path = "processed_data.xlsx"  # 替换为您的文件路径
data = pd.read_excel(file_path)

# Step 2: 获取第 H 到第 M 列
# 假设 H 到 M 是列索引的位置，例如 H = 7, M = 12（以 0 为起始索引）
columns_to_process = data.columns[7:13]  # 替换为实际的 H 到 M 列范围

# Step 3: 填充空值为非空值的平均值
for column in columns_to_process:
    mean_value = data[column].mean(skipna=True)  # 计算非空值的平均值
    data[column].fillna(mean_value, inplace=True)  # 用平均值填充空值

# Step 4: 保存处理后的数据
output_file_path = "filled_data.xlsx"  # 替换为您想保存的路径
data.to_excel(output_file_path, index=False)


In [6]:
unique_sources = data["新闻标题情绪标签"].dropna().unique()

# 创建来源类别映射表
source_mapping = {source: idx for idx, source in enumerate(unique_sources)}

# 新增“来源类别”列
data["新闻标题类别"] = data["新闻标题情绪标签"].map(source_mapping)
output_file_path = "filled_data.xlsx"  # 替换为您想保存的路径
data.to_excel(output_file_path, index=False)